# 0. Import/Classes/

In [1]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException, ElementNotInteractableException
import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle
from collections import deque
from tqdm import tqdm
import pandas as pd
import logging

In [2]:
# получение пользовательского логгера и установка уровня логирования
py_logger = logging.getLogger('test1')
py_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
py_handler = logging.FileHandler(f"{'test1'}.log", mode='w')
py_formatter = logging.Formatter("%(name)s %(asctime)s %(levelname)s %(message)s")

# добавление форматировщика к обработчику 
py_handler.setFormatter(py_formatter)
# добавление обработчика к логгеру
py_logger.addHandler(py_handler)

# получение пользовательского логгера и установка уровня логирования
py_logger2 = logging.getLogger('test2')
py_logger2.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
py_handler2 = logging.FileHandler(f"{'test2'}.log", mode='w')
py_formatter2 = logging.Formatter("%(name)s %(asctime)s %(levelname)s %(message)s")

# добавление форматировщика к обработчику 
py_handler2.setFormatter(py_formatter2)
# добавление обработчика к логгеру
py_logger2.addHandler(py_handler2)

In [2]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    page_href: str

class MyTimeoutErrore(Exception):
    def __init__(self, text):
        self.txt = text

In [87]:
driver = webdriver.Chrome()

In [3]:
try:
    raise TimeoutException
except Exception as ex:
    py_logger.error("TimeoutException")
    py_logger2.error("TimeoutException", exc_info=True)

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [84]:
def _get_page_count(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [85]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [86]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

list index out of range
list index out of range
list index out of range


['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [87]:
def _get_retailers_links(driver: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutErrore('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [88]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(_get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
# print(retailer_links)

55


### 2.2.3. Save/Load

In [89]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [90]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

# 3. Получение адресов Товаров

## 3.1. Получение ссылок на каталоги с товарами

### 3.1.1. Функции

In [92]:
def _get_page_count2(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> tuple[list[str],bool]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    
    empty_page = False
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            try:
                # Проверка на отсутствие скидок.
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
            else:
                empty_page = True
                out_list = None
                alcohol = True
                break
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol, empty_page)

def _get_all_links_to_market(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    dead_line = 100
    # Очередь
    q = deque(list_url)

    out_list = list()
    with tqdm(total=len(list_url)) as pbar:
        while len(q) > 0:
            if dead_line < 0:
                break
            
            item = q.popleft()
            
            if item.discounts > 24:
                try:
                    temp_url_list, alcohol, empty_page = _get_page_count2(driver, item.href, '?page=', timeout)
                except MyTimeoutErrore as mte:
                    q.append(item)
                    loger_fault.append(mte)
                    dead_line -= 1
                    continue
                if not empty_page:
                    for href in temp_url_list:
                        out_list.append(market_link(item.sity,
                                                    item.title,
                                                    item.discounts,
                                                    alcohol,
                                                    item.href,
                                                    href))
                    
            elif item.discounts:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            True,
                                            item.href,
                                            item.href))
            else:
                out_list.append(market_link(*item[:-1], None))
            
            pbar.update(1)

    return (out_list, loger_fault, q)

### 3.1.2. Выполнение

In [93]:
# Получение всех адресов на страницы с товарами.
retailer_links_pages, loger_fault, q = _get_all_links_to_market(driver, retailer_links, timeout=10)
len(retailer_links_pages)

100%|██████████| 55/55 [05:47<00:00,  6.32s/it]


907

### 3.1.3. Save/Load

In [94]:
with open('data/retailer_links_pages.pickle', 'wb') as file:
    pickle.dump(retailer_links_pages, file)

In [31]:
with open('data/retailer_links_pages.pickle', 'rb') as file:
    retailer_links_pages = pickle.load(file)

## 3.2. Получение информации о товарах и ссылки на них

### 3.2.1 Функции

In [167]:
def _get_page_count2(driver: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='grid']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"grid"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not GRID in url: {url}')
    
    return (out_list, alcohol)

def _get_all_links_to_market(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    # Очередь
    q = deque(list_url)
    # Нужно вывести дейдлайн, который позволит вывести обработаные данные, и не обработанные. ???????????
    out_list = list()
    while len(q) > 0:
        item = q.popleft()
        
        if item.discounts > 24:
            try:
                temp_url_list, alcohol = _get_page_count2(driver, item.href, '?page=', timeout)
            except MyTimeoutErrore as mte:
                q.append(item)
                loger_fault.append(mte)
                continue
            
            for href in temp_url_list:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            alcohol,
                                            item.href,
                                            href))
                
        elif item.discounts:
            out_list.append(market_link(item.sity,
                                        item.title,
                                        item.discounts,
                                        True,
                                        item.href,
                                        item.href))
        else:
            out_list.append(market_link(*item[:-1], None))

    return (out_list, loger_fault)



def _get_sku_info(driver: webdriver.Chrome, item: market_link, timeout: int=30):
    """
        Возвращает что-то об продукте.
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    if item.alcohol:
        # Проверка появления "Да, мне есть 18"
        try:
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        except TimeoutException:
            pass
        except ElementNotInteractableException:
            pass                                                                                # Нужно добавить логирование
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol)
    

### 3.2.2. Выполнение

#### 3.2.2.1. Сохранение всех страниц, для того что бы отточить работу парсера.

In [102]:
def _get_page(driver: webdriver.Chrome, item: market_link, timeout: int=30):
    """
        сохраняем страницы
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    if item.alcohol:
        # Проверка появления "Да, мне есть 18"
        try:
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        except TimeoutException:
            pass
        except ElementNotInteractableException:
            pass                                                                                # Нужно добавить логирование
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        # Варант провекрки загрузки страницы, проверка что текст не пустой

        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@data-test-ref='grid']"))
        except TimeoutException:
            try:
                driver.find_element(by=By.XPATH, value="//div[@class='b-dsk-stub__description']")
            except TimeoutException:
                raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
            else:
                page = None
                break
            
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            # Дожидаемся когда загрузится информация о первом товаре на странице
            f1 = page.find('div', {'data-test-ref':'grid'})
            f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
            f2[0].find('div', class_='b-srch-card__title').get_text(strip=True)
        except IndexError as ex:
            continue

        break
    
    else:
        raise MyTimeoutErrore(f'timeout not GRID in url: {url}')
    
    return driver.page_source

def _get_pages(driver: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение страниц
    """
    with tqdm(total=len(list_url)) as pbar:
        loger_fault = list()
        # Очередь
        q = deque(list_url)
        # Нужно вывести дейдлайн, который позволит вывести обработаные данные, и не обработанные. ???????????
        dead_line = 100
        out_list = list()
        while len(q) > 0:
            if dead_line < 0:
                return (out_list, loger_fault, q)
            
            item = q.popleft()
            if item.discounts:
                try:
                    page = _get_page(driver, item, timeout)
                except MyTimeoutErrore as mte:
                    q.append(item)
                    loger_fault.append(mte)
                    dead_line -= 1
                    continue
                
                out_list.append(page)
            pbar.update(1)
            
    return (out_list, loger_fault, q)

In [104]:
pages, loger_fault, q = _get_pages(driver, retailer_links_pages[:100], timeout=20)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [10:33<00:00,  6.33s/it]


In [105]:
# Сохранение страниц
with ZipFile('data/pages.zip', 'w', compression=ZIP_DEFLATED, compresslevel=1) as zip_file:
    for index, page in enumerate(pages):
        if page:
            zip_file.writestr(f'{index:05d}.html', page.encode())

In [ ]:
pages = list()
with ZipFile('data/pages.zip', 'r') as zf:
    for item in zf.filelist:
        with zf.open(item.filename) as file:
            pages.append(file.read())
return pages_list

In [32]:
url = retailer_links_pages[0].page_href

In [78]:
def _parser_sku_01(driver: webdriver.Chrome, market: market_link, date: datetime) -> pd.DataFrame:
    # Парсинг данных из файлов
    host = '/'.join(market.page_href.split('/', maxsplit=3)[:3])
    # 
    # page = BS(driver.page_source, 'lxml')
    page = driver
    f1 = page.find('div', {'data-test-ref':'grid'})
    f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
    
    test = list()
    
    for f3 in f2:
        # название
        title = f3.find('div', class_='b-srch-card__title').get_text(strip=True)
        # дата до истечения
        date_to = datetime.strptime(f3.find('time').get('datetime'), '%Y-%m-%d')
        # текущая валюта
        unit = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"currency"}).get_text(strip=True)
        # Цена (разбита на части)
        b = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"money-base"}).get_text(strip=True)
        s = f3.find('div', class_='b-money__root-wrapper').find('span', {'data-test-ref':"money-subunit"}).get_text(strip=True)
        price = float(f'{b}.{s}')
        # скидка
        sale = f3.find('div', class_='b-srch-card__discount-label').get_text(strip=True)
        # вес
        f4 = f3.find('div', class_='b-srch-card__info').find('span').get_text(strip=True).replace('\xa0', ' ').split('•')
        vol = f4[0].strip()
        # руб/кг. руб/шт.
        vol_vs = f4[1].strip()
        # ссылка
        href = host + f3.find('a', class_="b-srch-card__card-wrapper").get('href')
        
        test.append((date, market.title, title, date_to, price, unit, sale, vol, vol_vs, href))

    return pd.DataFrame(test)

dataframe_list = list()

for page in pages[:2]:
    dataframe_list.append(_parser_sku_01(page, retailer_links_pages[0], datetime.strptime("2023-08-28", '%Y-%m-%d')))
pd.concat(dataframe_list, ignore_index=True)

,0,1,2,3,4,5,6,7,8,9
0,2023-08-28,Пятёрочка,"Туалетная бумага Zewa, с ароматом персика, 3 с...",2023-08-28,179.90,₽,–51%,8 шт,"22,49 ₽/шт",https://edadeal.ru/lipeck/offers/e8cd559a-e727...
1,2023-08-28,Пятёрочка,"Туалетная бумага Zewa Deluxe Delicate Care, 3 ...",2023-08-28,179.90,₽,–51%,8 шт,"22,49 ₽/шт",https://edadeal.ru/lipeck/offers/354cd123-56dd...
2,2023-08-28,Пятёрочка,"Колбаса Салями Венская, полукопчёная, Останкин...",2023-08-28,119.90,₽,–50%,350 г,"342,57 ₽/кг",https://edadeal.ru/lipeck/offers/567434fa-15db...
3,2023-08-28,Пятёрочка,"Сок Nar, гранатовый, 0,31 л",2023-08-28,44.99,₽,–48%,"0,31 л","145,13 ₽/л",https://edadeal.ru/lipeck/offers/379d31b6-8f2b...
4,2023-08-28,Пятёрочка,"Пюре Фрутоняня, яблоко, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/metaoffers/3378b35b-...
5,2023-08-28,Пятёрочка,"Пюре Фрутоняня, груша, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/offers/fdb95367-e1cb...
6,2023-08-28,Пятёрочка,"Десерт Фрутоняня, яблоко-клубника-малина, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/offers/33eaae49-59ef...
7,2023-08-28,Пятёрочка,"Пюре Фрутоняня, витаминный салатик, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/offers/bf20b1b0-1ae4...
8,2023-08-28,Пятёрочка,"Пюре Фрутоняня, яблоко-манго-йогурт, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/offers/1ac482fc-7f16...
9,2023-08-28,Пятёрочка,"Пюре Фрутоняня, яблоко-абрикос-сливки, 90 г",2023-08-28,35.99,₽,–44%,90 г,"399,89 ₽/кг",https://edadeal.ru/lipeck/offers/150138de-3d07...
